# Similar mindset to create multiple wrg exports from different climatologies
## Prerequisite setup

Locate your Windsim Project and your WindSim installation

Install python with jupyter notebook installed,
or (recommended) use pipenv and the Pipfile to set up the environment.

* This assumes the project has been running up Windfields and an initial objects to configure the Layout
* You have measurements at different heights


In [46]:
# Python needs to be >= 3.9
from pathlib import Path
import sys
if sys.version_info < (3,8):
    print(f"Python version is {sys.version_info}, needs to be 3.9 or newer.")

from pathlib import Path    
import shutil
import xml.etree.ElementTree as ET
from typing import List
import itertools
import os
import time
import types
import subprocess
import datetime



## Project and environment variables
### System
* windsim: WindSim installation folder
* environment: Where your environment file is located
* layout: Name of the layout with suffix
### Project 
* project: Directory where your project lives
* owsfile: Name of the layout, containing the turbines you want to use as Actuator disks
* layout: Name of the layout with suffix


In [6]:
# Defining what you want to run
project = Path('C:/Users/GullikKillie/Documents/WindSim Projects 12/development/Hundhammer/Hundhammer.ws')
windsim=Path('C:/Program Files/WindSim/WindSim 12.0.0')
environment=Path('C:/Users/GullikKillie/AppData/Roaming/WindSim/1200/Environment.xml')
layout='Layout 1.lws'

## Project
Defining the project class, project specific information

In [71]:
# Defining the project we want to work with
class Project():
    def __init__(self, project_path: Path, layout: str="Layout 1"):
        self.name = project_path.stem
        self.project_file = project_path
        self.layout = layout
        self.base_directory = self.project_file.parent.parent

project = Project(project_path=Path('C:/Users/GullikKillie/Documents/WindSim Projects 12/development/Hundhammer/Hundhammer.ws'), layout='Layout 1')
climatologies =[{'name': 'Hundhammer_30m', 'height': 30}, {'name': 'Hundhammer_73m', 'height': 73}, {'name': 'Hundhammer_83m', 'height': 83}]


# LayoutCreator
Next we create the LayoutCreator, which will create a layout for each climatology, this can be adapted to do other mixes with turbines as well mixed in the layouts.

1. Copy the initial layout to layouts with climatology names
2. Change out height and names in layout files
3. Add entry into LayoutLists
4. Run Object for different climatologies
5. Run WindResources for different layouts/climatologies

In [75]:
class LayoutCreator:
    def __init__(self, project: Project, climatologies: list[str]) :
        self.project = project
        self.climatologies = climatologies


    def copy_layout(self, climatology: str) -> None: 
        """Copies a layout, layout folder and layout-file, this requires a layout to be present"""
        
        original_layout =self.project.project_file.parent / self.project.layout
        new_layout =self.project.project_file.parent / climatology
        print(f"Making new layout {climatology} using climatology {climatology}, in project {self.project.project_file.parent}")
        
        #Copy folder
        shutil.copytree(src=original_layout, dst=new_layout, dirs_exist_ok=True)

        # Copy layout-file
        shutil.copy(src=original_layout.with_suffix('.lws'), dst= new_layout.with_suffix('.lws'))

        # Add entry in layoutlist
        self._add_layout_entry(climatology=climatology)

        return 
    
    def _change_climatology(self, climatology:str, height:int):
        '''Changes the climatology in a layout'''
        ET.register_namespace('', 'LayoutParameters.xsd')
        layout_file =(self.project.project_file.parent / climatology).with_suffix('.lws')

        tree = ET.parse(layout_file)
        root = tree.getroot()
        namespace = {'layout_parameters': 'LayoutParameters.xsd'}

        for object in root.findall("{LayoutParameters.xsd}Objects"):
            for objectpoint in object.findall("{LayoutParameters.xsd}ObjectPoint"):            
                self._replace_or_add_element(namespace=namespace, field=objectpoint, parameter='IDText', value=climatology)
                self._replace_or_add_element(namespace=namespace, field=objectpoint, parameter='Zpos', value=height)
                self._replace_or_add_element(namespace=namespace, field=objectpoint, parameter='IDhtml', value=climatology)
                self._replace_or_add_element(namespace=namespace, field=objectpoint, parameter='WecsClimFileName', value=climatology)


        ET.indent(tree, '  ')
        tree.write(layout_file, encoding = "UTF-8",  xml_declaration=True)

        return
    
    def _add_layout_entry(self, climatology:str):
        ''' Adds the new entry in the LayoutList'''
        ET.register_namespace('', 'LayoutList.xsd')
        tree = ET.parse(self.project.project_file.parent / 'LayoutList.xml')
        root = tree.getroot()
        namespace = {'layoutlist_parameters': 'LayoutList.xsd'}

        # Create a new layout element
        new_layout = ET.Element("Layout")
        ET.SubElement(new_layout, "Name").text = climatology
        ET.SubElement(new_layout, "LastOpened").text = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f%z")
        ET.SubElement(new_layout, "ShortName").text = climatology
        ET.SubElement(new_layout, "ShortFullPath").text = (self.project.project_file.parent / climatology).with_suffix('.lws').as_posix().replace('/', '\\')
        # Append the new layout to the root element
        root.append(new_layout)

        # Save the modified XML file
        ET.indent(tree, '  ')

        tree.write(self.project.project_file.parent / 'LayoutList_new', encoding="utf-8", xml_declaration=True)


        return
    
    def _replace_or_add_element(self, namespace, field: ET.Element, parameter: str, value: any):
        # This may or may not be available
        parameter_present = False
        for child in field.findall("{LayoutParameters.xsd}"+parameter, namespace):
            parameter_present = True
            child.text = str(value)
        if not parameter_present:
            ows_element = ET.SubElement(field, parameter)
            ows_element.text = str(value)

layout_creator = LayoutCreator(project=project, climatologies=climatologies)

layout_creator.copy_layout(climatology='Hundhammer_83m')
layout_creator._add_layout_entry(climatology='Hundhammer_83m')
layout_creator._change_climatology(climatology='Hundhammer_83m', height=83)

Making new layout Hundhammer_83m using climatology Hundhammer_83m, in project C:\Users\GullikKillie\Documents\WindSim Projects 12\development\Hundhammer


In [ ]:
        #Changing the Number of nodes and cells in the project
        ET.register_namespace('', 'ProjectParameters.xsd')
        tree = ET.parse(dest_dir  / f"{self.project.name}.ws")
        root = tree.getroot()

        namespace = {'project_parameters': 'ProjectParameters.xsd'}

        for WindFields in root.findall("{ProjectParameters.xsd}WindField"):
            self._replace_or_add_element(namespace, WindFields, "VelocityBoundaryLayer", windspeed)

        ET.indent(tree, '  ')
        tree.write(dest_dir  / f"{self.project.name}.ws", encoding = "UTF-8",  xml_declaration=True)

In [16]:
def run_Object(self, name: str):
    os.makedirs('tmp/', exist_ok=True)
    os.chdir('tmp')
    
    print(f'Run Terrain for {name}')
    subprocess.run(f'"{self.windsim / "bin" / "Terrain.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}"')
        
    print(f'Run Report[Terrain] for {name}')

    subprocess.run(f'"{self.windsim / "bin" / "Reports.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}" 3')

    os.chdir('../')

    shutil.rmtree('tmp')


# Add Terrain runner to runner
runner.run_Terrains = types.MethodType(run_Terrains, runner)
runner.run_Terrain = types.MethodType(run_object, runner)

runner.run_Terrains()


NameError: name 'run_Terrains' is not defined